In [1]:
import numpy as np
from itertools import permutations, combinations, chain
import time
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix
from cvxopt import matrix, solvers, sparse, spmatrix
from numba import njit
import functools 
import copy

In [2]:
#shamelessly stolen from https://github.com/sympy/sympy/blob/master/sympy/combinatorics/perm_groups.py
@njit
def Is_vec_in_mat(vec,mat):
    assume=True
    for elem in mat:
        if np.array_equal(vec, elem):
            assume=False
            break
    return assume


@njit
def dimino_wolfe(group_generators):
        gens=group_generators
        degree=np.max(gens)+1
        idn = np.arange(degree)
        order = 0
        element_list = [idn]
        #element_list=np.atleast_2d(idn)
        #set_element_list = {tuple(idn)}
        for i in np.arange(len(gens)):
            # D elements of the subgroup G_i generated by gens[:i]
            D = element_list[:]
            N = [idn]
            while N:
                A = N
                N = []
                for a in A:
                    for g in gens[:i + 1]:
                        ag = a[g]
                        if Is_vec_in_mat(ag,element_list):
                        #if not np.any(np.all(ag==np.array(element_list,np.int64),axis=1)):
                        #if ag not in np.array(element_list):
                        #if tuple(ag) not in set_element_list:
                            # produce G_i*g
                            for d in D:
                                order += 1
                                ap = d[ag]
                                element_list.append(ap)
                                #element_list=np.append(element_list, np.atleast_2d(ap), axis=0)
                                #set_element_list.add(tuple(ap))
                                N.append(ap)
        return element_list
    
#np.array(dimino_wolfe(np.array([[1,0,2,3,4,5],[1,2,3,4,5,0]])))

In [3]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [4]:

#PROBLEM INPUT -- ASSUMED TO BE TOPOLOGICALLY SORTED GRAPH
graph_structure=[[0],[1,2],[1,3],[0,3]];
latent_count=2;
inflation_order=3;

In [5]:
#INFERRED PARAMETERS
obs_count=len(graph_structure)
#latent_count=0;
#for elem in graph_structure:
#    if len(elem) == 0:
#        latent_count+=1
#latent_count
#obs_count=len(graph_structure)-latent_count
#(latent_count,obs_count)

In [6]:
root_structure=copy.deepcopy(graph_structure)
highest_parent_val=max(list(map(max,root_structure)))
while highest_parent_val>=latent_count:
    subst=graph_structure[highest_parent_val-latent_count]
    for idx,elem in enumerate(root_structure):
        try:
            elem.remove(highest_parent_val)
            elem.extend(subst)
            root_structure[idx]=np.unique(elem).tolist()
            #root_structure[idx]=list(set(elem))
        except ValueError:
            pass
    #root_structure=list(map(compose(list,np.unique),root_structure))
    highest_parent_val=max(list(map(max,root_structure)))
root_structure 

[[0], [0, 1], [0, 1], [0, 1]]

In [7]:
inflation_depths=np.array(list(map(len,root_structure)))
inflation_depths

array([1, 2, 2, 2])

In [8]:

inflationcopies=inflation_order**inflation_depths
inflationcopies

array([3, 9, 9, 9], dtype=int32)

In [9]:
num_vars=inflationcopies.sum()
var_count=num_vars
num_vars

30

#### Code to extract expressible set (for any inflation order)

In [10]:
np.ravel_multi_index(np.diag_indices(3,2),(3,3))

array([0, 4, 8], dtype=int64)

In [11]:
accumulated=np.add.accumulate(inflationcopies)
accumulated

array([ 3, 12, 21, 30], dtype=int32)

In [12]:
diagonal_positions=np.array(list(np.ravel_multi_index(np.diag_indices(inflation_order, depth),tuple(np.full(depth,inflation_order))) for depth in inflation_depths))
accumulated=np.add.accumulate(inflationcopies)
offsets = np.hstack(([0],accumulated[:-1]))
exp_set=np.ravel(diagonal_positions.T+offsets)
exp_set

array([ 0,  3, 12, 21,  1,  7, 16, 25,  2, 11, 20, 29], dtype=int64)

#### Code to identify the generating symmetries (FIXED I THINK)

In [13]:
globalstrategyflat=list(np.add(*stuff) for stuff in zip(list(map(np.arange,inflationcopies.tolist())),offsets))
globalstrategyflat

[array([0, 1, 2]),
 array([ 3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([12, 13, 14, 15, 16, 17, 18, 19, 20]),
 array([21, 22, 23, 24, 25, 26, 27, 28, 29])]

In [14]:
reshapings=np.ones((obs_count,latent_count),np.uint8)
#reshapings=np.ones(inflation_order*obs_count,np.uint8).reshape((obs_count,inflation_order))
for idx,elem in enumerate(root_structure):
    reshapings[idx][elem]=inflation_order
reshapings=list(map(tuple,reshapings))
reshapings

[(3, 1), (3, 3), (3, 3), (3, 3)]

In [15]:
globalstrategyshaped=list(np.reshape(*stuff) for stuff in zip(globalstrategyflat,reshapings))
globalstrategyshaped

[array([[0],
        [1],
        [2]]), array([[ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]), array([[12, 13, 14],
        [15, 16, 17],
        [18, 19, 20]]), array([[21, 22, 23],
        [24, 25, 26],
        [27, 28, 29]])]

In [16]:
#I'm thinking we broadcast, take permutations, and thread.
fullshape=tuple(np.full(latent_count,inflation_order))
fullshape

(3, 3)

In [17]:
#@njit
def Deduplicate(ar): #Alternatives include unique_everseen and panda.unique, see https://stackoverflow.com/a/15637512 and https://stackoverflow.com/a/41577279
    (vals,idx)=np.unique(ar,return_index=True)
    return vals[np.argsort(idx)]


@njit
def MoveToFront(num_var,ar):
    return np.hstack((ar,np.delete(np.arange(num_var),ar)))

In [18]:
#Now that I've implemented some proper group theory functions, let's focus on obtaining minimal group representatives
if inflation_order==2:
    inflation_order_gen_count=1
else:
    inflation_order_gen_count=2
#inflation_order_gen_count=np.math.factorial(inflation_order)-1
group_generators=np.zeros((latent_count,inflation_order_gen_count,num_vars),np.uint)
for latent_to_explore in np.arange(latent_count):
    for gen_idx in np.arange(inflation_order_gen_count):
        initialtranspose=MoveToFront(latent_count,np.array([latent_to_explore]))
        #inversetranspose=np.hstack((np.array([1,0]),2+np.argsort(initialtranspose)))
        inversetranspose=np.hstack((np.array([0]),1+np.argsort(initialtranspose)))
        label_permutation=np.arange(inflation_order)
        if gen_idx==0:
            label_permutation[np.array([0,1])]=np.array([1,0])
        elif gen_idx==1:
            label_permutation=np.roll(label_permutation, 1)
        global_permutation=np.array(list(np.broadcast_to(elem,fullshape).transpose(tuple(initialtranspose))[label_permutation] for elem in globalstrategyshaped))
        global_permutation=np.transpose(global_permutation,tuple(inversetranspose))
        #global_permutation=np.reshape(global_permutation,(obs_count,-1))
        global_permutation=Deduplicate(np.ravel(global_permutation))
        group_generators[latent_to_explore,gen_idx]=global_permutation
        #group_generators[latent_to_explore,gen_idx]=np.array(list(map(Deduplicate,global_permutation)))
        #afterpermutationcalc=np.array(list(list(permutations(np.broadcast_to(elem,fullshape).transpose(tuple(initialtranspose))))[1:] for elem in globalstrategyshaped))
        #afterpermutationcalc=np.transpose(afterpermutationcalc,tuple(inversetranspose))
        #afterpermutationcalc=np.reshape(afterpermutationcalc,(afterpermutationcalc.shape[0],-1))
        #initmatrix[latent_to_explore]=np.array(list(map(Deduplicate,afterpermutationcalc)))
#group_generators=initmatrix.reshape((latent_count*inflation_order_gen_count,num_vars))
group_generators

array([[[ 1,  0,  2,  6,  7,  8,  3,  4,  5,  9, 10, 11, 15, 16, 17, 12,
         13, 14, 18, 19, 20, 24, 25, 26, 21, 22, 23, 27, 28, 29],
        [ 2,  0,  1,  9, 10, 11,  3,  4,  5,  6,  7,  8, 18, 19, 20, 12,
         13, 14, 15, 16, 17, 27, 28, 29, 21, 22, 23, 24, 25, 26]],

       [[ 0,  1,  2,  4,  3,  5,  7,  6,  8, 10,  9, 11, 13, 12, 14, 16,
         15, 17, 19, 18, 20, 22, 21, 23, 25, 24, 26, 28, 27, 29],
        [ 0,  1,  2,  5,  3,  4,  8,  6,  7, 11,  9, 10, 14, 12, 13, 17,
         15, 16, 20, 18, 19, 23, 21, 22, 26, 24, 25, 29, 27, 28]]],
      dtype=uint32)

In [19]:
(len(dimino_wolfe(group_generators[0])),
 len(dimino_wolfe(group_generators.reshape((-1,num_vars)))))

(6, 36)

In [20]:
"""
inflation_order_gen_count=np.math.factorial(inflation_order)-1
initmatrix=np.zeros((latent_count,inflation_order_gen_count,num_vars),np.uint)
for latent_to_explore in np.arange(latent_count):
    initialtranspose=MoveToFront(latent_count,np.array([latent_to_explore]))
    inversetranspose=np.hstack((np.array([1,0]),2+np.argsort(initialtranspose)))
    afterpermutationcalc=np.array(list(list(permutations(np.broadcast_to(elem,fullshape).transpose(tuple(initialtranspose))))[1:] for elem in globalstrategyshaped))
    afterpermutationcalc=np.transpose(afterpermutationcalc,tuple(inversetranspose))
    afterpermutationcalc=np.reshape(afterpermutationcalc,(afterpermutationcalc.shape[0],-1))
    initmatrix[latent_to_explore]=np.array(list(map(Deduplicate,afterpermutationcalc)))
group_generators=initmatrix.reshape((latent_count*inflation_order_gen_count,num_vars))
group_generators
""";

#### ToDo: Add code to identify determinism assumptions

In [21]:
print(graph_structure)

[[0], [1, 2], [1, 3], [0, 3]]


In [22]:
root_structure

[[0], [0, 1], [0, 1], [0, 1]]

In [23]:
#We can identify which latent-&-grandchild+ pairs require rule adjustment, this is based on contrasting graph_structure with root_structure.
pairs_to_adjust=list();
for idx,parents in enumerate(root_structure):
    for latent_parent in parents:
        if latent_parent not in graph_structure[idx]:
            print([latent_parent,idx])
            pairs_to_adjust.append((latent_parent,np.hstack((
                [pidx-latent_count for pidx in graph_structure[idx] if pidx>=latent_count and latent_parent in root_structure[pidx-latent_count]]
                ,[idx]))));
pairs_to_adjust

[0, 1]
[0, 2]
[1, 3]


[(0, array([0, 1])), (0, array([1, 2])), (1, array([1, 3]))]

In [24]:
list(permutations(np.arange(inflation_order)))[1:][np.math.factorial(inflation_order-1)-1]

(1, 0, 2)

In [25]:
#now, for every such pair we need to identify the parents of the observable variable which screen off the root variable...
#this is accomplished by returning all those indices which contiain the latent variable in their root structure

In [26]:
#UPDATE: Using the Dimino algorithm for enumerating group elements, we can now just stick the the first group generator for every latent variable.
#one_generator_per_root=group_generators.reshape((latent_count,inflation_order_gen_count,num_vars))[:,np.math.factorial(inflation_order-1)-1] #Has to use THAT ONE to get the  (1,0,2,3,4...) permutation!
one_generator_per_root=group_generators[:,0]
one_generator_per_root

array([[ 1,  0,  2,  6,  7,  8,  3,  4,  5,  9, 10, 11, 15, 16, 17, 12,
        13, 14, 18, 19, 20, 24, 25, 26, 21, 22, 23, 27, 28, 29],
       [ 0,  1,  2,  4,  3,  5,  7,  6,  8, 10,  9, 11, 13, 12, 14, 16,
        15, 17, 19, 18, 20, 22, 21, 23, 25, 24, 26, 28, 27, 29]],
      dtype=uint32)

In [27]:
det_assumptions=list();
for pair in pairs_to_adjust:
    flatset=exp_set[pair[1]]
    symop=one_generator_per_root[pair[0]]
    rule=np.vstack((flatset,symop[flatset])).T
    rule=rule[:-1,:].T.tolist()+rule[-1,:].T.tolist()
    det_assumptions.append(rule)
det_assumptions    
#list(np.array([exp_set[pair[1]],one_generator_per_root[pair[0],exp_set[pair[1]]]]).T for pair in pairs_to_adjust)

[[[0], [1], 3, 6], [[3], [6], 12, 15], [[3], [4], 21, 22]]

In [28]:
det_assumptions[0]

[[0], [1], 3, 6]